In [1]:
import numpy as np
import pandas as pd
#import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time
import collections
import itertools as it
from sklearn import ensemble, linear_model
import matplotlib.pyplot as plt

### Results from simulation experiment #1

- Setup 120 strategies, let 30 agents draw their strategy randomly, and simulate to find the on-average best response strategy

In [23]:
# Load simulation outputs from two different sources

id1 = pd.read_csv('output_simulation/s1_ids.csv', delimiter=' ', header=None, dtype=np.int)
id2 = pd.read_csv('output_simulation/s2_sams_ids.csv', delimiter=' ', header=None, dtype=np.int)
ids = pd.concat([id1,id2])
print(ids.shape)
ids.head()

(3526, 30)


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,128,123,145,15,74,47,89,179,84,92,...,56,167,76,173,133,0,108,123,165,143
1,84,173,93,132,45,10,119,121,157,89,...,165,82,177,95,146,27,56,24,56,46
2,171,123,37,134,55,127,58,171,143,138,...,122,96,28,27,32,149,147,149,81,161
3,80,52,79,43,71,70,123,76,144,172,...,121,175,65,170,76,109,36,161,79,98
4,44,98,176,122,88,16,26,157,133,17,...,153,73,65,23,33,4,4,110,173,52


In [24]:
clicks1 = pd.read_csv('output_simulation/s1_click.csv', delimiter=' ', header=None)
clicks2 = pd.read_csv('output_simulation/s2_sams_click.csv', delimiter=' ', header=None)
clicks = pd.concat([clicks1,clicks2])
print(ids.shape)
clicks.head()

(3526, 30)


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,11.0,2.0,3.0,0.0,2.0,17.0,4.0,0.0,2.0,8.0,...,1.0,2.0,0.0,0.0,4.0,57.0,15.0,2.0,0.0,0.0
1,1.0,0.0,1.0,7.0,0.0,3.0,16.0,5.0,0.0,0.0,...,1.0,44.0,0.0,0.0,2.0,27.0,0.0,7.0,0.0,0.0
2,0.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,2.0,6.0,6.0,23.0,7.0,1.0,5.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,5.0,0.0,4.0,3.0,...,9.0,1.0,14.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0
4,0.0,2.0,0.0,7.0,3.0,1.0,6.0,0.0,4.0,0.0,...,0.0,1.0,12.0,5.0,1.0,11.0,9.0,13.0,0.0,1.0


In [25]:
wins1 = pd.read_csv('output_simulation/s1_win.csv', delimiter=' ', header=None)
wins2 = pd.read_csv('output_simulation/s2_sams_win.csv', delimiter=' ', header=None)
wins = pd.concat([wins1,wins2])
print(wins.shape)
wins.head()

(3526, 30)


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,40467.0,7291.0,3473.0,8300.0,17286.0,6566.0,96.0,97.0,677.0,15528.0,...,10850.0,237.0,8532.0,408.0,14720.0,9000.0,16671.0,7280.0,212.0,3.0
1,8675.0,5322.0,5287.0,42438.0,20.0,9162.0,7774.0,14545.0,7.0,7.0,...,5393.0,14162.0,379.0,2927.0,13594.0,15876.0,3869.0,6296.0,4082.0,1335.0
2,0.0,8521.0,9044.0,10513.0,10910.0,7760.0,0.0,0.0,4119.0,10723.0,...,10266.0,23925.0,3582.0,15367.0,19863.0,701.0,11369.0,701.0,3925.0,2856.0
3,15853.0,5104.0,4866.0,3133.0,2389.0,5352.0,9828.0,788.0,19233.0,19465.0,...,16296.0,4739.0,2915.0,10953.0,617.0,1378.0,16535.0,4006.0,4889.0,1534.0
4,144.0,4526.0,4629.0,14024.0,1458.0,8464.0,9401.0,14.0,17983.0,7815.0,...,1195.0,6917.0,6787.0,18482.0,9583.0,2716.0,2760.0,14545.0,4595.0,8271.0


In [26]:
costs1 = pd.read_csv('output_simulation/s1_cost.csv', delimiter=' ', header=None)
costs2 = pd.read_csv('output_simulation/s2_sams_cost.csv', delimiter=' ', header=None)
costs = pd.concat([costs1,costs2])
print(costs.shape)
costs.head()

(3526, 30)


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,6248132.0,6249826.0,656837.0,520935.0,6249723.0,853552.0,25299.0,6257.0,108991.0,1558117.0,...,891889.0,28647.0,6249940.0,33971.0,6249600.0,3437645.0,2847278.0,6249511.0,16614.0,186.0
1,598298.0,463175.0,453306.0,5863942.0,1076.0,1205253.0,2540673.0,6249910.0,632.0,628.0,...,461422.0,1858316.0,25264.0,171743.0,1636364.0,4159662.0,323956.0,1251587.0,343159.0,143377.0
2,0.0,6249167.0,6249215.0,6249681.0,707221.0,6249657.0,0.0,0.0,629472.0,6249388.0,...,6249103.0,2170013.0,1001108.0,3985835.0,6249608.0,124478.0,1168819.0,123592.0,635799.0,168033.0
3,1212232.0,383749.0,6249694.0,511600.0,6247248.0,6249161.0,6249789.0,5902334.0,2194542.0,1698008.0,...,6249772.0,405408.0,2279586.0,600680.0,6221197.0,303072.0,6249919.0,240487.0,6249521.0,111755.0
4,7051.0,351918.0,360649.0,6249334.0,154205.0,627797.0,2012928.0,1265.0,6249618.0,407073.0,...,147424.0,6249411.0,2644054.0,3639439.0,6249583.0,1174087.0,1223642.0,2501712.0,379043.0,617232.0


In [27]:
total_appearances = np.zeros(shape=(ids.shape[0], 180))
total_clicks = np.zeros(shape=(ids.shape[0], 180))
total_wins = np.zeros(shape=(ids.shape[0], 180))
total_costs = np.zeros(shape=(ids.shape[0], 180))

for row_id, row in ids.iterrows():       
    strategy_id = row 
    total_appearances[row_id, strategy_id] = 1
    total_clicks[row_id, strategy_id] = clicks.iloc[row_id, :]
    total_wins[row_id, strategy_id] = wins.iloc[row_id, :]
    total_costs[row_id, strategy_id] = costs.iloc[row_id, :]

In [28]:
total_appearances = pd.DataFrame(total_appearances)
total_clicks = pd.DataFrame(total_clicks)
total_wins = pd.DataFrame(total_wins)
total_costs = pd.DataFrame(total_costs)

# Gather everything
total_res = pd.DataFrame(np.zeros(shape=(180,5)), columns=['id', 'n', 'clicks', 'wins', 'costs'])
total_res.iloc[:,0] = total_res.index
total_res.iloc[:,1] = total_appearances.sum()
total_res.iloc[:,2] = total_clicks.sum()/total_appearances.sum()
total_res.iloc[:,3] = total_wins.sum()/total_appearances.sum()
total_res.iloc[:,4] = total_costs.sum()/total_appearances.sum()

strategy_names = pd.read_csv('proper_datasets/bid_look_up.csv', nrows=0, index_col=0).columns

total_res.set_index(strategy_names, inplace=True)
total_res.sort_values(by='clicks', ascending=False).head(20)

,id,n,clicks,wins,costs
NN_linbid_calm,66,526.0,32.237643,10965.074144,3.777785e+06
NN_linbid_aggre,67,496.0,30.274194,8610.344758,3.422973e+06
Damian_linbid,1,446.0,27.103139,9359.778027,3.774564e+06
Damian_linbid_aggre,63,501.0,27.087824,9636.864271,3.796332e+06
NN_linbid,3,484.0,25.758264,8303.330579,3.079531e+06
Damian_linbid_calm,62,512.0,25.337891,11364.664062,3.847462e+06
ensemble1_linbid_calm,68,522.0,19.511494,7534.593870,2.469201e+06
ensemble1_linbid_aggre,69,554.0,19.361011,7283.606498,2.317124e+06
ensemble1_linbid,4,508.0,17.112205,6436.877953,2.160118e+06
Stefan2_linbid_calm,64,527.0,12.836812,9486.736243,2.768685e+06


### Results from simulation experiment #2

- Setup 120 strategies, let 30 agents draw their strategy randomly, and simulate to find the on-average best response strategy.
- Each round of simulation is computed on a bootstrap sample, i.e. a random sample of original length drawn with replacement (some impressions will be drawn multiple times)

In [29]:
ids = pd.read_csv('output_simulation/s2_bt_ids.csv', delimiter=' ', header=None, dtype=np.int)
clicks = pd.read_csv('output_simulation/s2_bt_click.csv', delimiter=' ', header=None)
wins = pd.read_csv('output_simulation/s2_bt_win.csv', delimiter=' ', header=None)
costs = pd.read_csv('output_simulation/s2_bt_cost.csv', delimiter=' ', header=None)
print(costs.shape)

total_appearances = np.zeros(shape=(ids.shape[0], 180))
total_clicks = np.zeros(shape=(ids.shape[0], 180))
total_wins = np.zeros(shape=(ids.shape[0], 180))
total_costs = np.zeros(shape=(ids.shape[0], 180))

for row_id, row in ids.iterrows():
    strategy_id = row 
    total_appearances[row_id, strategy_id] = 1
    total_clicks[row_id, strategy_id] = clicks.iloc[row_id, :]
    total_wins[row_id, strategy_id] = wins.iloc[row_id, :]
    total_costs[row_id, strategy_id] = costs.iloc[row_id, :]
    
total_appearances = pd.DataFrame(total_appearances)
total_clicks = pd.DataFrame(total_clicks)
total_wins = pd.DataFrame(total_wins)
total_costs = pd.DataFrame(total_costs)

# Gather everything
total_res = pd.DataFrame(np.zeros(shape=(180,5)), columns=['id', 'n', 'clicks', 'wins', 'costs'])
total_res.iloc[:,0] = total_res.index
total_res.iloc[:,1] = total_appearances.sum()
total_res.iloc[:,2] = total_clicks.sum()/total_appearances.sum()
total_res.iloc[:,3] = total_wins.sum()/total_appearances.sum()
total_res.iloc[:,4] = total_costs.sum()/total_appearances.sum()

strategy_names = pd.read_csv('proper_datasets/bid_look_up.csv', nrows=0, index_col=0).columns

total_res.set_index(strategy_names, inplace=True)
total_res.sort_values(by='clicks', ascending=False).head(20)

(2283, 30)


,id,n,clicks,wins,costs
ensemble1_revlinbid_cap1,38,350.0,19.220000,31902.951429,6.177515e+06
Stefan2_revlinbid_cap1,34,360.0,17.897222,28902.133333,6.217534e+06
ensemble1_revlinbid_cap1_aggre,137,349.0,15.661891,26241.361032,6.199647e+06
ensemble1_revlinbid_cap1_calm,136,369.0,15.474255,25810.848238,6.195897e+06
Stefan2_revlinbid_cap1_calm,128,337.0,14.430267,23561.433234,6.227526e+06
Stefan2_revlinbid_cap1_aggre,129,352.0,14.340909,23484.224432,6.231667e+06
Damian_revlinbid_cap1,32,348.0,13.752874,23222.847701,6.199196e+06
log_reg_revlinbid_cap1,30,386.0,12.476684,19919.512953,6.231575e+06
Damian_revlinbid_cap1_aggre,125,330.0,12.121212,19791.769697,6.220308e+06
NN_revlinbid_cap1,36,356.0,11.727528,18833.601124,6.238236e+06


### Results from simulation experiment #3

- Take out the most simple 20 of the previous 120 strategies, let 29 agents draw their strategy randomly, and simulate the best on-average (multi-agent) bidding fit for the ORTB2 strategy.
- Each round of simulation is computed on a bootstrap sample, i.e. a random sample of original length drawn with replacement (some impressions will be drawn multiple times)
- The simulation experiment is repeated using four different CTR estimators: LR, GBRT1, GBRT2, Ensemble
- In total 50 different bootstrap samples (the ORTB2 parameters have been tuned on each and averaged)

In [30]:
# Load CTR estimators
CTR_models_val = pd.DataFrame(np.vstack([np.loadtxt('CTR_models/logr/r1_sams_logr_6.csv'),
                                        np.loadtxt('CTR_models/gbm_dam/pCTR_gbm_dam_val.csv'), 
                                        pd.read_csv('CTR_models/gbm2/pCTR_gbm2_val.csv', index_col=0).values.flatten(), 
                                        np.loadtxt('ensemble/ensemble_base_learners_set2/val/r7_mlp_nn_14.csv')]).T,
                            columns=['log_reg', 'Damian', 'Stefan2', 'NN'])

# Build ensemble
weighted_CTR_val = CTR_models_val[['Damian', 'Stefan2', 'NN']].copy()

w = np.atleast_2d([1, 6, 5]) #np.array([x1, x2, x3])
weighted_CTR_val = np.log(weighted_CTR_val/(1 + weighted_CTR_val))
weighted_CTR_val =  np.dot(weighted_CTR_val, w.T)
weighted_CTR_val /= np.sum(w) # normalize weights
weighted_CTR_val = 1 / (1 + np.exp(-weighted_CTR_val)) # take logistic / sigmoid
weighted_CTR_val = weighted_CTR_val.flatten()

CTR_models_val['ensemble1'] = weighted_CTR_val
CTR_models_val.head()

,log_reg,Damian,Stefan2,NN,ensemble1
0,0.000533,0.000078,0.000291,0.000155,0.000200
1,0.000205,0.000093,0.000211,0.000017,0.000069
2,0.000139,0.000063,0.000351,0.000011,0.000072
3,0.000210,0.000075,0.000274,0.000025,0.000091
4,0.000306,0.000092,0.000335,0.000007,0.000061


In [31]:
############# CTR: LR #############

res_bt_logreg = pd.read_csv('output_simulation/ortb_multi/r1_acer_logreg.csv', index_col=0)
print(res_bt_logreg.shape)

tmp_str = res_bt_logreg['3'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_bt_logreg.shape[0])]
res_bt_logreg['4'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_bt_logreg.shape[0])]
res_bt_logreg['5'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_bt_logreg.shape[0])]
res_bt_logreg['6'] = pd.Series(costs_l, dtype=np.float32)

res_bt_logreg.drop(columns='3', inplace=True)
res_bt_logreg.columns = ['bt_id', 'cc', 'lamb', 'clicks', 'wins', 'costs']

res_bt_logreg.groupby(['cc','lamb']).mean().sort_values(by='clicks', ascending=False).head(20)

(2850, 4)


bt_id     clicks          wins      costs
cc lamb                                                   
58 1.034793e-06   24.5  38.720001  75980.179688  6132269.0
   8.280345e-07   24.5  36.660000  73218.242188  6249948.5
20 1.000000e-08   24.5  36.599998  50977.898438  6249923.5
58 7.763448e-07   24.5  36.340000  72150.156250  6249947.0
   9.831034e-07   24.5  36.299999  76394.703125  6249960.0
70 1.000000e-06   24.5  36.119999  71349.460938  6249948.5
58 1.086483e-06   24.5  36.099998  71524.703125  5692954.0
   7.246552e-07   24.5  36.099998  71007.078125  6249953.0
70 1.000000e-07   24.5  36.020000  50158.218750  6249910.5
60 1.000000e-07   24.5  36.020000  51878.140625  6249934.5
50 1.000000e-07   24.5  35.919998  54105.019531  6249931.5
58 1.043793e-07   24.5  35.880001  52500.019531  6249912.0
   9.314138e-07   24.5  35.740002  75318.476562  6249951.5
30 1.000000e-07   24.5  35.580002  62148.820312  6249939.0
58 8.797241e-07   24.5  35.480000  74247.562500  6249950.0
60 1.000000e-06   24.5  35.299999  75703.539062  6249953.0
58 6.729655e-07   24.5  35.180000  69903.617188  6249953.0
   1.560690e-07   24.5  34.900002  55242.859375  6249917.0
20 1.000000e-07   24.5  34.840000  71187.882812  6249954.5
58 5.268966e-08   24.5  34.799999  48651.941406  6249910.5

In [32]:
############# CTR: GBRT1 #############

res_bt_dam = pd.read_csv('output_simulation/ortb_multi/r1_acer_dam.csv', index_col=0)
print(res_bt_dam.shape)

tmp_str = res_bt_dam['3'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_bt_dam.shape[0])]
res_bt_dam['4'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_bt_dam.shape[0])]
res_bt_dam['5'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_bt_dam.shape[0])]
res_bt_dam['6'] = pd.Series(costs_l, dtype=np.float32)

res_bt_dam.drop(columns='3', inplace=True)
res_bt_dam.columns = ['bt_id', 'cc', 'lamb', 'clicks', 'wins', 'costs']

res_bt_dam.groupby(['cc','lamb']).mean().sort_values(by='clicks', ascending=False).head(20)

(2850, 4)


bt_id     clicks          wins      costs
cc lamb                                                   
58 5.268966e-08   24.5  37.180000  50937.218750  6249909.0
   6.729655e-07   24.5  36.980000  69917.960938  6249950.5
20 1.000000e-08   24.5  36.740002  53291.039062  6249930.0
58 8.280345e-07   24.5  36.740002  71519.460938  6166874.0
60 1.000000e-07   24.5  36.580002  54710.398438  6249918.5
58 6.212759e-07   24.5  36.560001  69023.562500  6249942.5
90 1.000000e-07   24.5  36.320000  49938.500000  6249909.5
80 1.000000e-07   24.5  36.320000  51329.199219  6249900.5
70 1.000000e-06   24.5  36.279999  70082.023438  6235342.5
58 1.043793e-07   24.5  36.200001  55386.460938  6249939.0
50 1.000000e-07   24.5  36.099998  56952.500000  6249935.5
70 1.000000e-07   24.5  36.060001  52828.218750  6249925.0
58 7.246552e-07   24.5  35.860001  70795.781250  6249945.5
20 1.000000e-07   24.5  35.840000  73277.843750  6249947.5
58 7.763448e-07   24.5  35.820000  71582.578125  6249949.0
10 1.000000e-08   24.5  35.720001  63451.859375  6249944.5
58 8.797241e-07   24.5  35.380001  66689.156250  5686506.5
   5.695862e-07   24.5  34.959999  67984.921875  6249954.0
80 1.000000e-06   24.5  34.919998  67218.757812  6249946.0
58 2.594483e-07   24.5  34.840000  62009.140625  6249939.5

In [33]:
############# CTR: GBRT2 #############

res_bt_gbm2 = pd.read_csv('output_simulation/ortb_multi/r1_sams_gbm2.csv', index_col=0)
print(res_bt_gbm2.shape)

tmp_str = res_bt_gbm2['3'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_bt_gbm2.shape[0])]
res_bt_gbm2['4'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_bt_gbm2.shape[0])]
res_bt_gbm2['5'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_bt_gbm2.shape[0])]
res_bt_gbm2['6'] = pd.Series(costs_l, dtype=np.float32)

res_bt_gbm2.drop(columns='3', inplace=True)
res_bt_gbm2.columns = ['bt_id', 'cc', 'lamb', 'clicks', 'wins', 'costs']

res_bt_gbm2.groupby(['cc','lamb']).mean().sort_values(by='clicks', ascending=False).head(20)

(2850, 4)


bt_id     clicks          wins      costs
cc lamb                                                   
58 9.831034e-07   24.5  40.759998  82429.960938  6231628.0
60 1.000000e-06   24.5  39.360001  81917.781250  6249951.0
58 8.280345e-07   24.5  39.320000  78730.703125  6249961.0
70 1.000000e-06   24.5  38.520000  77091.140625  6249948.0
58 7.763448e-07   24.5  38.380001  77387.320312  6249956.0
   8.797241e-07   24.5  38.340000  80042.523438  6249953.0
   9.314138e-07   24.5  38.320000  81332.156250  6249957.5
60 1.000000e-07   24.5  38.160000  52103.398438  6249907.0
58 1.034793e-06   24.5  37.779999  76956.960938  5725429.5
   1.560690e-07   24.5  37.700001  55933.820312  6249935.0
50 1.000000e-07   24.5  37.599998  54483.781250  6249939.5
58 1.043793e-07   24.5  37.500000  52804.281250  6249924.5
20 1.000000e-08   24.5  37.459999  50954.281250  6249922.0
58 7.246552e-07   24.5  37.320000  76015.796875  6249957.0
40 1.000000e-07   24.5  37.180000  57872.761719  6249929.5
58 2.077586e-07   24.5  36.860001  58540.378906  6249930.0
70 1.000000e-07   24.5  36.459999  50424.820312  6249917.5
58 3.628276e-07   24.5  36.400002  64928.460938  6249952.0
20 1.000000e-07   24.5  36.279999  73479.539062  6249945.5
58 4.145172e-07   24.5  36.200001  66812.960938  6249938.5

In [36]:
############# CTR: Ensemble #############

res_bt_ens1 = pd.read_csv('output_simulation/ortb_multi/r1_sams_ens1.csv', index_col=0)
print(res_bt_ens1.shape)

tmp_str = res_bt_ens1['3'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_bt_ens1.shape[0])]
res_bt_ens1['4'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_bt_ens1.shape[0])]
res_bt_ens1['5'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_bt_ens1.shape[0])]
res_bt_ens1['6'] = pd.Series(costs_l, dtype=np.float32)

res_bt_ens1.drop(columns='3', inplace=True)
res_bt_ens1.columns = ['bt_id', 'cc', 'lamb', 'clicks', 'wins', 'costs']

res_bt_ens1.groupby(['cc','lamb']).mean().sort_values(by='clicks', ascending=False).head(20)

(2850, 4)


bt_id     clicks          wins      costs
cc lamb                                                   
20 1.000000e-08   24.5  36.700001  52038.218750  6249929.5
60 1.000000e-07   24.5  36.380001  53250.000000  6249931.0
70 1.000000e-07   24.5  36.340000  51473.800781  6249916.5
58 1.043793e-07   24.5  36.299999  53987.941406  6249916.0
50 1.000000e-07   24.5  36.080002  55720.058594  6249937.0
80 1.000000e-07   24.5  35.939999  50050.140625  6249905.0
58 1.560690e-07   24.5  35.740002  57147.339844  6249934.0
   5.268966e-08   24.5  35.459999  49754.500000  6249904.0
   6.212759e-07   24.5  35.060001  70960.921875  6249953.5
20 1.000000e-07   24.5  35.020000  74385.843750  6249958.0
58 7.763448e-07   24.5  34.919998  73835.679688  6249953.5
   6.729655e-07   24.5  34.540001  71956.023438  6249952.5
30 1.000000e-07   24.5  34.439999  64671.339844  6249944.5
40 1.000000e-07   24.5  34.139999  59276.218750  6249940.0
58 3.111379e-07   24.5  34.060001  63367.238281  6249949.0
   5.695862e-07   24.5  34.000000  69889.882812  6249954.0
   2.594483e-07   24.5  33.900002  61687.960938  6249940.5
   7.246552e-07   24.5  33.840000  72866.781250  6249950.0
90 1.000000e-07   24.5  33.799999  48878.980469  6249895.5
10 1.000000e-08   24.5  33.799999  63141.261719  6249942.0

### Results from simulation experiment #3.5

- Same as experiment #3, only using linear bidding strategy rather than ORTB2.
- Take out the most simple 20 of the previous 120 strategies, let 29 agents draw their strategy randomly, and simulate the best on-average (multi-agent) bidding fit for the <b>linear</b> strategy.
- Each round of simulation is computed on a bootstrap sample, i.e. a random sample of original length drawn with replacement (some impressions will be drawn multiple times)
- The simulation experiment is repeated using just one CTR estimator: Ensemble
- In total 50 different bootstrap samples (the base_bid and cap parameters have been tuned on each and averaged)

In [60]:

res_time_linbid_ens1 = pd.read_csv('output_simulation/ortb_multi/r4_linbid_acer_ens1.csv', index_col=0)
print(res_time_linbid_ens1.shape)

tmp_str = res_time_linbid_ens1['3'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_time_linbid_ens1.shape[0])]
res_time_linbid_ens1['4'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_time_linbid_ens1.shape[0])]
res_time_linbid_ens1['5'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_time_linbid_ens1.shape[0])]
res_time_linbid_ens1['6'] = pd.Series(costs_l, dtype=np.float32)

res_time_linbid_ens1.drop(columns='3', inplace=True)
res_time_linbid_ens1.columns = ['bt_id', 'base_bid', 'cap', 'clicks', 'wins', 'costs']

res_time_linbid_ens1['aCPC'] = res_time_linbid_ens1['costs'] / res_time_linbid_ens1['clicks'] /1000
res_time_linbid_ens1['aCPM'] = res_time_linbid_ens1['costs'] / res_time_linbid_ens1['wins'] /1000

res_time_linbid_ens1.groupby(['base_bid', 'cap']).mean().sort_values(by='clicks', ascending=False).head(10)

(4950, 4)


,,bt_id,clicks,wins,costs,aCPC,aCPM
base_bid,cap,,,,,,
240.0,1000,24.5,26.700001,42372.238281,6249909.5,236.519852,0.147523
235.0,1000,24.5,26.379999,42335.980469,6249873.0,239.283127,0.147648
230.0,1000,24.5,26.120001,42288.000000,6249889.0,241.916412,0.147816
225.0,1000,24.5,25.900000,42241.058594,6249853.5,244.114075,0.147979
220.0,1000,24.5,25.520000,42197.398438,6249895.0,248.370621,0.148133
240.0,3000,24.5,25.100000,39019.101562,6249814.5,252.066025,0.160217
235.0,3000,24.5,25.080000,38983.738281,6249859.0,251.867828,0.160362
215.0,1000,24.5,24.980000,42157.558594,6249828.0,254.207779,0.148271
225.0,3000,24.5,24.940001,38875.859375,6249854.5,253.688980,0.160810


In [61]:
res_time_linbid_ens1.groupby(['base_bid', 'cap']).mean().sort_values(by='aCPC', ascending=True).head(10)

,,bt_id,clicks,wins,costs,aCPC,aCPM
base_bid,cap,,,,,,
240.0,1000,24.5,26.700001,42372.238281,6249909.5,236.519852,0.147523
235.0,1000,24.5,26.379999,42335.980469,6249873.0,239.283127,0.147648
230.0,1000,24.5,26.120001,42288.000000,6249889.0,241.916412,0.147816
225.0,1000,24.5,25.900000,42241.058594,6249853.5,244.114075,0.147979
220.0,1000,24.5,25.520000,42197.398438,6249895.0,248.370621,0.148133
235.0,3000,24.5,25.080000,38983.738281,6249859.0,251.867828,0.160362
240.0,3000,24.5,25.100000,39019.101562,6249814.5,252.066025,0.160217
225.0,3000,24.5,24.940001,38875.859375,6249854.5,253.688980,0.160810
230.0,3000,24.5,24.879999,38928.421875,6249754.0,253.989761,0.160588


In [62]:
res_time_linbid_ens1.groupby(['base_bid', 'cap']).mean().sort_values(by='aCPM', ascending=True).head(10)

,,bt_id,clicks,wins,costs,aCPC,aCPM
base_bid,cap,,,,,,
240.0,1000,24.5,26.700001,42372.238281,6249909.5,236.519852,0.147523
235.0,1000,24.5,26.379999,42335.980469,6249873.0,239.283127,0.147648
230.0,1000,24.5,26.120001,42288.000000,6249889.0,241.916412,0.147816
225.0,1000,24.5,25.900000,42241.058594,6249853.5,244.114075,0.147979
220.0,1000,24.5,25.520000,42197.398438,6249895.0,248.370621,0.148133
215.0,1000,24.5,24.980000,42157.558594,6249828.0,254.207779,0.148271
210.0,1000,24.5,24.379999,42109.761719,6249874.0,260.484161,0.148439
205.0,1000,24.5,23.879999,42061.960938,6249891.0,265.937958,0.148608
200.0,1000,24.5,23.900000,42005.480469,6249884.0,265.550446,0.148807


### Results from simulation experiment #4

- Take out the most simple 20 of the previous 120 strategies, let 29 agents draw their strategy randomly, and simulate the best on-average (multi-agent) time preference settings out of 49 possible.
- Each round of simulation is computed on a bootstrap sample, i.e. a random sample of original length drawn with replacement (some impressions will be drawn multiple times).
- The simulation experiment is repeated using four different ORTB2 parameter combinations (best ones from comparing results #3 and leaderboard criteron #2) with just one CTR estimator: Ensemble.
- In total 50 different bootstrap samples (the ORTB2 parameters have been tuned on each and averaged)

In [41]:
############# CTR: Ensemble, ORTB2_parameter_setting: 1 #############

res_time_ens1_param1 = pd.read_csv('output_simulation/ortb_multi/r3_time_acer_ens1_params1.csv', index_col=0)
print(res_time_ens1_param1.shape)

tmp_str = res_time_ens1_param1['2'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_time_ens1_param1.shape[0])]
res_time_ens1_param1['3'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_time_ens1_param1.shape[0])]
res_time_ens1_param1['4'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_time_ens1_param1.shape[0])]
res_time_ens1_param1['5'] = pd.Series(costs_l, dtype=np.float32)

res_time_ens1_param1.drop(columns='2', inplace=True)
res_time_ens1_param1.columns = ['bt_id', 'time', 'clicks', 'wins', 'costs']

res_time_ens1_param1['aCPC'] = res_time_ens1_param1['costs'] / res_time_ens1_param1['clicks'] /1000
res_time_ens1_param1['aCPM'] = res_time_ens1_param1['costs'] / res_time_ens1_param1['wins'] /1000



(2500, 3)


In [42]:
res_time_ens1_param1.groupby(['time']).mean().sort_values(by='clicks', ascending=False).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t4_step7,24.5,37.500000,56237.460938,6249937.0,167.350021,0.111139
t3_step7,24.5,37.340000,54435.039062,6249922.0,167.893555,0.114819
t4_step6,24.5,37.220001,55807.160156,6249927.5,168.480865,0.111996
t4_step1,24.5,37.200001,54944.218750,6249940.5,168.594955,0.113756
t4_step3,24.5,37.160000,55156.578125,6249918.5,168.715240,0.113318
regular,24.5,36.880001,52092.039062,6249926.0,169.842911,0.119984
t3_step6,24.5,36.840000,53685.960938,6249920.0,170.177719,0.116422
t4_step5,24.5,36.799999,54408.699219,6249938.5,170.300583,0.114876
t4_step2,24.5,36.759998,53551.339844,6249931.5,170.564743,0.116717


In [43]:
res_time_ens1_param1.groupby(['time']).mean().sort_values(by='aCPC', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t4_step7,24.5,37.500000,56237.460938,6249937.0,167.350021,0.111139
t3_step7,24.5,37.340000,54435.039062,6249922.0,167.893555,0.114819
t4_step6,24.5,37.220001,55807.160156,6249927.5,168.480865,0.111996
t4_step1,24.5,37.200001,54944.218750,6249940.5,168.594955,0.113756
t4_step3,24.5,37.160000,55156.578125,6249918.5,168.715240,0.113318
regular,24.5,36.880001,52092.039062,6249926.0,169.842911,0.119984
t3_step6,24.5,36.840000,53685.960938,6249920.0,170.177719,0.116422
t4_step5,24.5,36.799999,54408.699219,6249938.5,170.300583,0.114876
t4_step2,24.5,36.759998,53551.339844,6249931.5,170.564743,0.116717


In [44]:
res_time_ens1_param1.groupby(['time']).mean().sort_values(by='aCPM', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t6_step7,24.5,34.759998,67619.679688,6249946.00,182.071533,0.092433
t7_step7,24.5,15.080000,26797.720703,2483696.25,174.101257,0.092733
t6_step6,24.5,33.820000,66870.140625,6249941.50,187.107285,0.093469
t6_step3,24.5,33.580002,65703.796875,6249946.00,188.478607,0.095128
t2_step7,24.5,34.759998,65680.117188,6249952.00,181.728104,0.095163
t6_step1,24.5,33.779999,65378.859375,6249938.00,187.361755,0.095601
t7_step6,24.5,18.740000,31992.919922,3086456.50,170.986526,0.096508
t2_step6,24.5,35.020000,64577.960938,6249952.00,180.332657,0.096786
t6_step5,24.5,34.220001,64355.921875,6249940.00,184.889389,0.097121


In [45]:
############# CTR: Ensemble, ORTB2_parameter_setting: 2 #############

res_time_ens1_param2 = pd.read_csv('output_simulation/ortb_multi/r3_time_acer_ens1_params2.csv', index_col=0)
print(res_time_ens1_param2.shape)

tmp_str = res_time_ens1_param2['2'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_time_ens1_param2.shape[0])]
res_time_ens1_param2['3'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_time_ens1_param2.shape[0])]
res_time_ens1_param2['4'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_time_ens1_param2.shape[0])]
res_time_ens1_param2['5'] = pd.Series(costs_l, dtype=np.float32)

res_time_ens1_param2.drop(columns='2', inplace=True)
res_time_ens1_param2.columns = ['bt_id', 'time', 'clicks', 'wins', 'costs']

res_time_ens1_param2['aCPC'] = res_time_ens1_param2['costs'] / res_time_ens1_param2['clicks'] /1000
res_time_ens1_param2['aCPM'] = res_time_ens1_param2['costs'] / res_time_ens1_param2['wins'] /1000

res_time_ens1_param2.groupby(['time']).mean().sort_values(by='clicks', ascending=False).head(10)

(2500, 3)


,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
regular,24.5,36.860001,51519.601562,6249922.5,169.982941,0.121317
t4_step7,24.5,36.240002,55415.781250,6249939.0,173.252502,0.112787
t3_step7,24.5,36.119999,53700.519531,6249917.0,173.770432,0.116390
t4_step6,24.5,35.759998,55000.421875,6249918.5,175.514450,0.113639
t4_step3,24.5,35.759998,54358.941406,6249897.5,175.617813,0.114980
t3_step6,24.5,35.700001,52960.980469,6249934.5,175.963089,0.118016
t4_step1,24.5,35.580002,54161.980469,6249928.5,176.596512,0.115399
t4_step4,24.5,35.320000,53198.699219,6249905.0,177.755173,0.117490
t3_step2,24.5,35.259998,49389.480469,6249882.0,178.109741,0.126554


In [46]:
res_time_ens1_param2.groupby(['time']).mean().sort_values(by='aCPC', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
regular,24.5,36.860001,51519.601562,6249922.5,169.982941,0.121317
t4_step7,24.5,36.240002,55415.781250,6249939.0,173.252502,0.112787
t3_step7,24.5,36.119999,53700.519531,6249917.0,173.770432,0.116390
t4_step6,24.5,35.759998,55000.421875,6249918.5,175.514450,0.113639
t4_step3,24.5,35.759998,54358.941406,6249897.5,175.617813,0.114980
t3_step6,24.5,35.700001,52960.980469,6249934.5,175.963089,0.118016
t4_step1,24.5,35.580002,54161.980469,6249928.5,176.596512,0.115399
t4_step4,24.5,35.320000,53198.699219,6249905.0,177.755173,0.117490
t3_step5,24.5,35.259998,50670.621094,6249925.0,178.088242,0.123353


In [47]:
res_time_ens1_param2.groupby(['time']).mean().sort_values(by='aCPM', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t6_step7,24.5,33.560001,65989.218750,6249946.5,188.229263,0.094716
t7_step7,24.5,15.000000,26419.400391,2513976.5,182.225693,0.095224
t6_step6,24.5,32.820000,65305.761719,6249952.5,192.868774,0.095707
t6_step3,24.5,32.740002,64192.359375,6249940.0,193.330811,0.097367
t2_step7,24.5,34.020000,64175.378906,6249945.0,185.918594,0.097393
t6_step1,24.5,33.119999,63869.039062,6249945.5,191.025681,0.097860
t2_step6,24.5,34.340000,63176.621094,6249940.0,183.899368,0.098932
t7_step6,24.5,18.100000,31566.480469,3121777.5,180.600006,0.098942
t6_step5,24.5,33.220001,62886.699219,6249941.5,190.163345,0.099389


In [49]:
############# CTR: Ensemble, ORTB2_parameter_setting: 3 #############

res_time_ens1_param3 = pd.read_csv('output_simulation/ortb_multi/r3_time_acer_ens1_params3.csv', index_col=0)
res_time_ens1_param3.head()

tmp_str = res_time_ens1_param3['2'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_time_ens1_param3.shape[0])]
res_time_ens1_param3['3'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_time_ens1_param3.shape[0])]
res_time_ens1_param3['4'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_time_ens1_param3.shape[0])]
res_time_ens1_param3['5'] = pd.Series(costs_l, dtype=np.float32)

res_time_ens1_param3.drop(columns='2', inplace=True)
res_time_ens1_param3.columns = ['bt_id', 'time', 'clicks', 'wins', 'costs']

res_time_ens1_param3['aCPC'] = res_time_ens1_param3['costs'] / res_time_ens1_param3['clicks'] /1000
res_time_ens1_param3['aCPM'] = res_time_ens1_param3['costs'] / res_time_ens1_param3['wins'] /1000

res_time_ens1_param3.groupby(['time']).mean().sort_values(by='clicks', ascending=False).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t4_step6,24.5,36.939999,74550.718750,6004174.5,165.599457,0.080566
t3_step6,24.5,36.660000,73075.742188,6249960.0,173.405853,0.085539
t3_step7,24.5,36.279999,75136.656250,6249951.5,175.276520,0.083194
t1_step5,24.5,35.540001,65340.878906,6249932.5,179.266403,0.095657
t1_step3,24.5,35.419998,66830.218750,6133876.5,176.921661,0.091793
t4_step5,24.5,35.060001,69403.742188,6249949.0,184.280807,0.090068
t4_step1,24.5,35.060001,72045.257812,6249953.5,183.246002,0.086767
t4_step3,24.5,34.820000,72927.976562,6249957.5,184.394196,0.085717
t3_step3,24.5,34.759998,67800.960938,6249953.5,182.871033,0.092191


In [50]:
res_time_ens1_param3.groupby(['time']).mean().sort_values(by='aCPC', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t4_step7,24.5,34.419998,70123.976562,5507461.00,163.308456,0.078580
t4_step6,24.5,36.939999,74550.718750,6004174.50,165.599457,0.080566
t1_step7,24.5,16.820000,34615.558594,2709578.25,167.685287,0.078318
t1_step6,24.5,22.260000,45847.320312,3676063.50,170.968109,0.080205
t2_step7,24.5,8.860000,17660.179688,1371475.25,172.251907,0.077739
t2_step6,24.5,11.660000,23266.759766,1854403.50,172.771027,0.079752
t3_step6,24.5,36.660000,73075.742188,6249960.00,173.405853,0.085539
t5_step7,24.5,23.680000,47967.980469,3983586.00,174.251266,0.083075
t3_step7,24.5,36.279999,75136.656250,6249951.50,175.276520,0.083194


In [51]:
res_time_ens1_param3.groupby(['time']).mean().sort_values(by='aCPM', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t7_step7,24.5,0.740000,1802.319946,1.173931e+05,inf,0.065253
t7_step6,24.5,1.320000,3410.800049,2.318812e+05,inf,0.068044
t6_step7,24.5,5.280000,10792.440430,8.287257e+05,200.349686,0.076932
t2_step7,24.5,8.860000,17660.179688,1.371475e+06,172.251907,0.077739
t1_step7,24.5,16.820000,34615.558594,2.709578e+06,167.685287,0.078318
t4_step7,24.5,34.419998,70123.976562,5.507461e+06,163.308456,0.078580
t6_step6,24.5,7.180000,14260.120117,1.128430e+06,177.115067,0.079222
t2_step6,24.5,11.660000,23266.759766,1.854404e+06,172.771027,0.079752
t1_step6,24.5,22.260000,45847.320312,3.676064e+06,170.968109,0.080205


In [52]:
############# CTR: Ensemble, ORTB2_parameter_setting: 4 #############

res_time_ens1_param4 = pd.read_csv('output_simulation/ortb_multi/r3_time_acer_ens1_params4.csv', index_col=0)
res_time_ens1_param4.head()

tmp_str = res_time_ens1_param4['2'].str.strip('()').str.split(',')
clicks_l= [tmp_str[i][0] for i in range(res_time_ens1_param4.shape[0])]
res_time_ens1_param4['3'] = pd.Series(clicks_l, dtype=np.float32)

wins_l= [tmp_str[i][1] for i in range(res_time_ens1_param4.shape[0])]
res_time_ens1_param4['4'] = pd.Series(wins_l, dtype=np.float32)

costs_l= [tmp_str[i][2] for i in range(res_time_ens1_param4.shape[0])]
res_time_ens1_param4['5'] = pd.Series(costs_l, dtype=np.float32)

res_time_ens1_param4.drop(columns='2', inplace=True)
res_time_ens1_param4.columns = ['bt_id', 'time', 'clicks', 'wins', 'costs']

res_time_ens1_param4['aCPC'] = res_time_ens1_param4['costs'] / res_time_ens1_param4['clicks'] /1000
res_time_ens1_param4['aCPM'] = res_time_ens1_param4['costs'] / res_time_ens1_param4['wins'] /1000

res_time_ens1_param4.groupby(['time']).mean().sort_values(by='clicks', ascending=False).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t4_step3,24.5,35.320000,66611.101562,6150548.0,179.317215,0.092358
t3_step6,24.5,34.939999,68297.578125,6249954.5,183.088699,0.091521
t1_step1,24.5,34.680000,63406.601562,6234332.5,185.976151,0.098328
t4_step1,24.5,34.680000,66899.882812,6249939.5,185.496292,0.093437
regular,24.5,34.340000,69840.757812,6249947.0,185.286209,0.089497
t3_step7,24.5,34.279999,70030.882812,6249956.5,186.829239,0.089257
t1_step5,24.5,34.020000,62024.539062,6249945.5,187.666077,0.100769
t4_step5,24.5,33.419998,64612.679688,6249941.0,191.769028,0.096743
t4_step2,24.5,33.360001,59413.820312,6249938.5,191.258224,0.105208


In [53]:
res_time_ens1_param4.groupby(['time']).mean().sort_values(by='aCPC', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t4_step6,24.5,31.240000,60961.480469,5302549.50,175.052094,0.087011
t4_step7,24.5,28.440001,56309.820312,4781112.50,175.184052,0.084941
t1_step6,24.5,19.680000,37933.339844,3281189.25,175.195877,0.086506
t1_step7,24.5,14.440000,28069.800781,2379721.25,177.441437,0.084793
t4_step3,24.5,35.320000,66611.101562,6150548.00,179.317215,0.092358
t1_step3,24.5,32.540001,58766.480469,5726931.00,182.213333,0.097453
t3_step6,24.5,34.939999,68297.578125,6249954.50,183.088699,0.091521
t5_step7,24.5,20.680000,40002.199219,3574018.00,183.183960,0.089356
t5_step6,24.5,23.320000,44651.121094,4095336.25,183.786972,0.091726


In [54]:
res_time_ens1_param4.groupby(['time']).mean().sort_values(by='aCPM', ascending=True).head(10)

,bt_id,clicks,wins,costs,aCPC,aCPM
time,,,,,,
t7_step7,24.5,0.480000,1185.140015,8.520286e+04,inf,0.072008
t7_step6,24.5,1.080000,2466.500000,1.823787e+05,inf,0.074000
t2_step7,24.5,7.600000,14118.440430,1.196125e+06,188.168030,0.084735
t1_step7,24.5,14.440000,28069.800781,2.379721e+06,177.441437,0.084793
t6_step7,24.5,4.500000,8471.700195,7.186283e+05,inf,0.084842
t4_step7,24.5,28.440001,56309.820312,4.781112e+06,175.184052,0.084941
t2_step6,24.5,10.280000,19109.199219,1.652150e+06,186.186966,0.086466
t6_step6,24.5,6.260000,11524.679688,9.967858e+05,200.872971,0.086500
t1_step6,24.5,19.680000,37933.339844,3.281189e+06,175.195877,0.086506
